Importing Libraries

In [1]:
!pip install pyspark
!pip install keras
!pip install tweet-preprocessor
!pip install vaderSentiment
import pyspark as spark
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,udf,monotonically_increasing_id,unix_timestamp,round,avg
import re
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 45.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=39b839ca0bc74ec6a58c2ec4b7c6e7ed1470d6ec2315267f98a641dbc29d25f8
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 15.2 MB/s 


Loading Tweet Dataset

In [13]:
from pyspark.sql.functions import *
from pyspark.sql import functions as sf
TwDF=pd.read_csv('sentiment_preprocessed.csv',error_bad_lines=False,engine = 'python',header = None)
TwDF = TwDF.drop([0], axis=1)
FullDataTw=sql.createDataFrame(TwDF)
FullDataTw = FullDataTw.dropna()
FullDataTw.select(monotonically_increasing_id().alias("rowId"),"*")
FullDataTw = FullDataTw.withColumnRenamed('1', 'DateTime')
FullDataTw = FullDataTw.withColumnRenamed('2', 'Date')
FullDataTw=FullDataTw.withColumn('Date', regexp_replace('Date', '/', '-'))
FullDataTw = FullDataTw.withColumnRenamed('3', 'Time') #setting column names of Twitter dataset
FullDataTw = FullDataTw.withColumnRenamed('4', 'Tweet')
FullDataTw =FullDataTw.withColumn('DateTime', sf.concat(sf.col('Date'),sf.lit(' '),sf.col('Time')))
FullDataTw=FullDataTw.filter(FullDataTw.Date!='1')
FullDataTw=FullDataTw.drop("Time")
Tw_samp = FullDataTw
Tw_samp.show(5)

+-------------------+----------+--------------------+
|           DateTime|      Date|               Tweet|
+-------------------+----------+--------------------+
|2022-01-31 23:59:57|2022-01-31|#Bitcoin  is one ...|
|2022-01-31 23:59:52|2022-01-31|Lol... Oh, you ju...|
|2022-01-31 23:59:46|2022-01-31|One of the larges...|
|2022-01-31 23:59:33|2022-01-31|However, if the r...|
|2022-01-31 23:59:33|2022-01-31|The CoinShares re...|
+-------------------+----------+--------------------+
only showing top 5 rows



Pre-Processing Twitter dataframe

In [14]:
import preprocessor as p #cleaning each tweet using tweet-preprocessor like removing hashtags,urls,emojis....
def function_udf(input_str):
    input_str = re.sub(r'RT', '', input_str)
    p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION)
    input_str = p.clean(input_str)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", input_str).split())
func_udf = udf(function_udf, StringType())
CleanDF = Tw_samp.withColumn('CleanedTweets', func_udf(Tw_samp['Tweet']))
CleanDF.show(3)


+-------------------+----------+--------------------+--------------------+
|           DateTime|      Date|               Tweet|       CleanedTweets|
+-------------------+----------+--------------------+--------------------+
|2022-01-31 23:59:57|2022-01-31|#Bitcoin  is one ...|Bitcoin is one of...|
|2022-01-31 23:59:52|2022-01-31|Lol... Oh, you ju...|Lol Oh you just r...|
|2022-01-31 23:59:46|2022-01-31|One of the larges...|One of the larges...|
+-------------------+----------+--------------------+--------------------+
only showing top 3 rows



Sentiment analysis using Vader packages

In [15]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
def senti_score_udf(sentence):
    snt = analyser.polarity_scores(sentence)
    return ([snt['neg'], snt['neu'], snt['pos'], snt['compound']])
func_udf2 = udf(senti_score_udf, ArrayType(FloatType()))
CleanDF = CleanDF.withColumn('p_neg', func_udf2(CleanDF['CleanedTweets'])[0])
CleanDF = CleanDF.withColumn('p_neu', func_udf2(CleanDF['CleanedTweets'])[1])
CleanDF = CleanDF.withColumn('p_pos', func_udf2(CleanDF['CleanedTweets'])[2])
CleanDF = CleanDF.withColumn('p_comp', func_udf2(CleanDF['CleanedTweets'])[3])
CleanDF = CleanDF.withColumn("DateTime",CleanDF['DateTime'].cast(TimestampType()))
CleanDF.show(3)

+-------------------+----------+--------------------+--------------------+-----+-----+-----+-------+
|           DateTime|      Date|               Tweet|       CleanedTweets|p_neg|p_neu|p_pos| p_comp|
+-------------------+----------+--------------------+--------------------+-----+-----+-----+-------+
|2022-01-31 23:59:57|2022-01-31|#Bitcoin  is one ...|Bitcoin is one of...|  0.0|0.809|0.191| 0.8555|
|2022-01-31 23:59:52|2022-01-31|Lol... Oh, you ju...|Lol Oh you just r...|  0.0|0.608|0.392| 0.7003|
|2022-01-31 23:59:46|2022-01-31|One of the larges...|One of the larges...|0.244|0.659|0.097|-0.5859|
+-------------------+----------+--------------------+--------------------+-----+-----+-----+-------+
only showing top 3 rows



Selecting necessary columns

In [16]:
FinalTw = CleanDF.selectExpr("DateTime as Date_Time","Date as Date", "CleanedTweets as Cleaned_Tweets", "p_neg","p_neu","p_pos","p_comp")
FinalTw.show(5) #selecting necessary columns

+-------------------+----------+--------------------+-----+-----+-----+-------+
|          Date_Time|      Date|      Cleaned_Tweets|p_neg|p_neu|p_pos| p_comp|
+-------------------+----------+--------------------+-----+-----+-----+-------+
|2022-01-31 23:59:57|2022-01-31|Bitcoin is one of...|  0.0|0.809|0.191| 0.8555|
|2022-01-31 23:59:52|2022-01-31|Lol Oh you just r...|  0.0|0.608|0.392| 0.7003|
|2022-01-31 23:59:46|2022-01-31|One of the larges...|0.244|0.659|0.097|-0.5859|
|2022-01-31 23:59:33|2022-01-31|However if the re...|  0.0|  1.0|  0.0|    0.0|
|2022-01-31 23:59:33|2022-01-31|The CoinShares re...|0.058|0.942|  0.0| -0.296|
+-------------------+----------+--------------------+-----+-----+-----+-------+
only showing top 5 rows



Truncating timestamps to hours and then grouping them by hour

In [ ]:
dt_truncated = ((round(unix_timestamp(col('Date_Time')) / 3600) * 3600).cast('timestamp'))
FinalTw = FinalTw.withColumn('dt_truncated', dt_truncated)
FinalTw = FinalTw.selectExpr("dt_truncated as Date_Time","Cleaned_Tweets","p_neg","p_neu","p_pos","p_comp")
UTC = ((unix_timestamp(col('Date_Time'))+ 5*60*60).cast('timestamp'))
FinalTw = FinalTw.withColumn('UTC', UTC)
FinalTw = FinalTw.selectExpr("UTC as Date_Time","Cleaned_Tweets","p_neg","p_neu","p_pos","p_comp")
FinalTw.show(5)

Taking average of days

In [17]:
FinalTw.drop("Date_Time")
FinalTw.registerTempTable("temp")
FinalTw_avg = sql.sql("SELECT Date As DateTime,AVG(p_neg) as P_Neg,AVG(p_neu) as P_Neu,AVG(p_pos) as P_Pos,AVG(p_comp) as P_Comp FROM temp GROUP BY Date")
#FinalTw_avg = FinalTw.select("Date_Time","polarity","subj","p_pos","p_neg").groupBy("Date_Time").agg(avg(col("polarity","subj","p_pos","p_neg")))
FinalTw_avg.show(5)

+----------+--------------------+------------------+-------------------+-------------------+
|  DateTime|               P_Neg|             P_Neu|              P_Pos|             P_Comp|
+----------+--------------------+------------------+-------------------+-------------------+
|2022-05-17|0.009156249929219484|0.8687187507748604|0.12215624912641943|0.25801249488722533|
|2022-03-30| 0.04526760607538089|0.8738309841760448|0.08088732358645385|0.08610422334725588|
|2022-01-20| 0.05092105249825277|0.8320789446956233|0.11707894827582334|0.13804473522070207|
|2022-07-04|  0.0558899077458666|0.8130733942219971| 0.1310733939529559| 0.2161155941494561|
|2022-07-08| 0.02996296242431358|0.8743703718538638|0.09566666765345468|0.23285555018594972|
+----------+--------------------+------------------+-------------------+-------------------+
only showing top 5 rows



This cell is just to collect all the corpus per hour(for the future work)

In [ ]:

from pyspark.sql import functions as f
df_with_text = FinalTw.groupby("Date_Time").agg(f.concat_ws(" ", f.collect_list(FinalTw.Cleaned_Tweets)))
df_with_text.show(1)


Sorting by time

In [18]:
FinalTw_avg.count()
from pyspark.sql.functions import *
df_sort = FinalTw_avg.sort(asc("DateTime"))
df_sort.show(5)

+----------+--------------------+------------------+-------------------+-------------------+
|  DateTime|               P_Neg|             P_Neu|              P_Pos|             P_Comp|
+----------+--------------------+------------------+-------------------+-------------------+
|2022-01-01|  0.0377575760763703|0.8556363636797125|0.10663636356140628|0.18419696785735362|
|2022-01-02| 0.05392307702165384|0.8503076938482431|0.09576923084946778|0.08094615374620144|
|2022-01-03| 0.07260606234723871|0.8002121204679663|0.12709090813542856|0.10791515299316609|
|2022-01-04|0.046699999943375585|0.8399600023031235|0.11336000017821789|0.12076200090348721|
|2022-01-05| 0.04544117696145002| 0.839235296144205| 0.1153529422248111| 0.2100970550056766|
+----------+--------------------+------------------+-------------------+-------------------+
only showing top 5 rows



Loading BTC Dataset

In [19]:
BtcDF=pd.read_csv('BTC-USD.csv',error_bad_lines=False,engine = 'python',header = None) 
FullDataBtc=sql.createDataFrame(BtcDF)
FullDataBtc = FullDataBtc.withColumnRenamed('0', 'DateTime')
FullDataBtc = FullDataBtc.withColumnRenamed('1', 'Open')
FullDataBtc = FullDataBtc.withColumnRenamed('2', 'High')
FullDataBtc = FullDataBtc.withColumnRenamed('3', 'Low')
FullDataBtc = FullDataBtc.withColumnRenamed('4', 'Close')
FullDataBtc = FullDataBtc.withColumnRenamed('5', 'Price')
FullDataBtc = FullDataBtc.withColumnRenamed('6', 'Volume')
FullDataBtc=FullDataBtc.filter(FullDataBtc.DateTime!='Date')
FullDataBtc=FullDataBtc.drop("Open","High","Low","Close","Volume")
FullDataBtc.show(5)

+----------+------------+
|  DateTime|       Price|
+----------+------------+
|2022-01-01|47686.812500|
|2022-01-02|47345.218750|
|2022-01-03|46458.117188|
|2022-01-04|45897.574219|
|2022-01-05|43569.003906|
+----------+------------+
only showing top 5 rows



Loading Nasdaq(Stock) Dataset

In [29]:
nasdaqDF=pd.read_csv('nasdaq2022.csv',error_bad_lines=False,engine = 'python',header = None) 
FullDatanasdaq=sql.createDataFrame(nasdaqDF)
FullDatanasdaq = FullDatanasdaq.withColumnRenamed('0', 'DateTime')
FullDatanasdaq = FullDatanasdaq.withColumnRenamed('1', 'Open')
FullDatanasdaq = FullDatanasdaq.withColumnRenamed('2', 'High')
FullDatanasdaq = FullDatanasdaq.withColumnRenamed('3', 'Low')
FullDatanasdaq = FullDatanasdaq.withColumnRenamed('4', 'Close')
FullDatanasdaq = FullDatanasdaq.withColumnRenamed('5', 'Price')
FullDatanasdaq = FullDatanasdaq.withColumnRenamed('6', 'Volume')
FullDatanasdaq=FullDatanasdaq.filter(FullDatanasdaq.DateTime!='Date')
FullDatanasdaq=FullDatanasdaq.drop("Open","High","Low","Close","Volume")
FullDatanasdaq.show(5)

+----------+------------+
|  DateTime|       Price|
+----------+------------+
|2022-01-03|15832.799805|
|2022-01-04|15622.719727|
|2022-01-05|15100.169922|
|2022-01-06|15080.860352|
|2022-01-07|14935.900391|
+----------+------------+
only showing top 5 rows



Joining twitter and bitcoin dataframes by DateTime

In [41]:
df_sort.registerTempTable("avgs")
FullDataBtc.registerTempTable("prices")
FullDatanasdaq.registerTempTable("nasdaqprices")

resultsBTC = sql.sql("SELECT nasdaqprices.DateTime, P_Neg, P_Neu, P_Pos, P_Comp, prices.Price FROM avgs JOIN prices ON avgs.DateTime = prices.DateTime JOIN nasdaqprices ON nasdaqprices.DateTime = prices.DateTime order by avgs.DateTime")
#results = results.selectExpr("DateTime","avg(polarity)","avg(subj)","avg(p_pos)","avg(p_neg)","Price") Use this line if you are using text blob package
resultsBTC.show(5)

+----------+--------------------+------------------+-------------------+-------------------+------------+
|  DateTime|               P_Neg|             P_Neu|              P_Pos|             P_Comp|       Price|
+----------+--------------------+------------------+-------------------+-------------------+------------+
|2022-01-03| 0.07260606234723871|0.8002121204679663|0.12709090813542856|0.10791515299316609|46458.117188|
|2022-01-04|0.046699999943375585|0.8399600023031235|0.11336000017821789|0.12076200090348721|45897.574219|
|2022-01-05| 0.04544117696145002| 0.839235296144205| 0.1153529422248111| 0.2100970550056766|43569.003906|
|2022-01-06|  0.0707727274434133| 0.841568182815205| 0.0875909089280123|0.06692272873426025|43160.929688|
|2022-01-07| 0.06091891903732274|0.8430810828466673|0.09600000073378151| 0.0921486464102526|41557.902344|
+----------+--------------------+------------------+-------------------+-------------------+------------+
only showing top 5 rows



In [42]:
resultsNasdaq = sql.sql("SELECT avgs.DateTime, P_Neg, P_Neu, P_Pos, P_Comp, nasdaqprices.Price FROM avgs JOIN nasdaqprices ON avgs.DateTime = nasdaqprices.DateTime order by avgs.DateTime")
#results = results.selectExpr("DateTime","avg(polarity)","avg(subj)","avg(p_pos)","avg(p_neg)","Price") Use this line if you are using text blob package
resultsNasdaq.show(5)

+----------+--------------------+------------------+-------------------+-------------------+------------+
|  DateTime|               P_Neg|             P_Neu|              P_Pos|             P_Comp|       Price|
+----------+--------------------+------------------+-------------------+-------------------+------------+
|2022-01-03| 0.07260606234723871|0.8002121204679663|0.12709090813542856|0.10791515299316609|15832.799805|
|2022-01-04|0.046699999943375585|0.8399600023031235|0.11336000017821789|0.12076200090348721|15622.719727|
|2022-01-05| 0.04544117696145002| 0.839235296144205| 0.1153529422248111| 0.2100970550056766|15100.169922|
|2022-01-06|  0.0707727274434133| 0.841568182815205| 0.0875909089280123|0.06692272873426025|15080.860352|
|2022-01-07| 0.06091891903732274|0.8430810828466673|0.09600000073378151| 0.0921486464102526|14935.900391|
+----------+--------------------+------------------+-------------------+-------------------+------------+
only showing top 5 rows



In [43]:
resultsNasdeqBTC = sql.sql("SELECT nasdaqprices.DateTime, P_Neg, P_Neu, P_Pos, P_Comp, prices.Price, nasdaqprices.Price as nasdaqPrice FROM avgs JOIN prices ON avgs.DateTime = prices.DateTime JOIN nasdaqprices ON nasdaqprices.DateTime = prices.DateTime order by avgs.DateTime")
#results = results.selectExpr("DateTime","avg(polarity)","avg(subj)","avg(p_pos)","avg(p_neg)","Price") Use this line if you are using text blob package
resultsNasdeqBTC.show(5)

+----------+--------------------+------------------+-------------------+-------------------+------------+------------+
|  DateTime|               P_Neg|             P_Neu|              P_Pos|             P_Comp|       Price| nasdaqPrice|
+----------+--------------------+------------------+-------------------+-------------------+------------+------------+
|2022-01-03| 0.07260606234723871|0.8002121204679663|0.12709090813542856|0.10791515299316609|46458.117188|15832.799805|
|2022-01-04|0.046699999943375585|0.8399600023031235|0.11336000017821789|0.12076200090348721|45897.574219|15622.719727|
|2022-01-05| 0.04544117696145002| 0.839235296144205| 0.1153529422248111| 0.2100970550056766|43569.003906|15100.169922|
|2022-01-06|  0.0707727274434133| 0.841568182815205| 0.0875909089280123|0.06692272873426025|43160.929688|15080.860352|
|2022-01-07| 0.06091891903732274|0.8430810828466673|0.09600000073378151| 0.0921486464102526|41557.902344|14935.900391|
+----------+--------------------+---------------

In [45]:
resultsBTC.write.csv("Final-BTC-Tweet.csv")
resultsNasdaq.write.csv("Final-Nasdaq-Tweet.csv")
resultsNasdeqBTC.write.csv("Final-Nasdaq-btc-Tweet.csv")

Save csv

In [ ]:
resultsBTC.repartition(1).write.csv("DataforModelExec.csv") #this will write df to single csv instead of writing diff csv acc to partitions 